
## <p style="text-align: center;">MIS 284N - Big Data and Distributed Programming</p>
## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and AWS</p>
## <p style="text-align: center;">Total points: 100</p>
## <p style="text-align: center;">Due: Thursday, October 19 submitted via Canvas by 11:59 pm</p>

Your project should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTEID for both students.

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

This project is on Tensorflow, its usage, Cloud services and why they are helpful in analyzing Big Data.

In this Project, we are trying to analyze crime data. This data is taken from <a href=https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2/data> here </a>. You have to download data from here, it should be around 1.6 GB in size. 

You have to predict type of crime that can happen given the features.

You have to complete all the four tasks and report your observations for each if any. There is a bonus task as well, it has two options and you can choose any one of it.

# Task 1 - Read the data and filter the data you need

This is open ended, you can use any features you want and drop any one of them. The 'Primary Type' is our target variable which we will be predicting for.
Split the data into training and testing in the ratio of 80:20.

In [1]:
import tensorflow as tf
import numpy as np

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd

In [3]:
data=pd.read_csv("Crimes-2001-present.csv")

In [4]:
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,...,28.0,25.0,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,...,15.0,67.0,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)"
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,4.0,39.0,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)"
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,...,3.0,40.0,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)"
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,28.0,25.0,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)"


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720963 entries, 0 to 6720962
Data columns (total 22 columns):
ID                      int64
Case Number             object
Date                    object
Block                   object
IUCR                    object
Primary Type            object
Description             object
Location Description    object
Arrest                  bool
Domestic                bool
Beat                    int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                object
X Coordinate            float64
Y Coordinate            float64
Year                    int64
Updated On              object
Latitude                float64
Longitude               float64
Location                object
dtypes: bool(2), float64(7), int64(3), object(10)
memory usage: 1.0+ GB


In [6]:
data.isnull().sum()


ID                           0
Case Number                  4
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description      3944
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614853
Community Area          616028
FBI Code                     0
X Coordinate             60187
Y Coordinate             60187
Year                         0
Updated On                   0
Latitude                 60187
Longitude                60187
Location                 60187
dtype: int64

In [7]:
data.dropna(axis=0, how='any', inplace=True)

In [8]:
data['date_parsed'] = pd.to_datetime(data['Date'])

In [9]:
data["dayofweek"]=data['date_parsed'].dt.weekday

In [10]:
data["hour"]=data['date_parsed'].dt.hour

In [11]:
data["Primary Type"].unique()

array(['BATTERY', 'OTHER OFFENSE', 'ROBBERY', 'NARCOTICS',
       'CRIMINAL DAMAGE', 'WEAPONS VIOLATION', 'THEFT', 'BURGLARY',
       'MOTOR VEHICLE THEFT', 'PUBLIC PEACE VIOLATION', 'ASSAULT',
       'CRIMINAL TRESPASS', 'CRIM SEXUAL ASSAULT',
       'INTERFERENCE WITH PUBLIC OFFICER', 'ARSON', 'DECEPTIVE PRACTICE',
       'LIQUOR LAW VIOLATION', 'KIDNAPPING', 'SEX OFFENSE',
       'OFFENSE INVOLVING CHILDREN', 'PROSTITUTION', 'HOMICIDE',
       'GAMBLING', 'INTIMIDATION', 'STALKING', 'OBSCENITY',
       'PUBLIC INDECENCY', 'HUMAN TRAFFICKING',
       'CONCEALED CARRY LICENSE VIOLATION', 'OTHER NARCOTIC VIOLATION',
       'NON - CRIMINAL', 'NON-CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'RITUALISM'], dtype=object)

In [12]:
data["Primary Type"]=data["Primary Type"].replace({'NON - CRIMINAL': 'NON-CRIMINAL'})
data["Primary Type"]=data["Primary Type"].replace({'NON-CRIMINAL (SUBJECT SPECIFIED)': 'NON-CRIMINAL'})

In [13]:
label=data["Primary Type"]

In [14]:
data["District"].unique()

array([11.,  7.,  2.,  9.,  5.,  6.,  4., 12., 15., 14., 20., 10., 22.,
        8., 17.,  3., 18., 16., 25.,  1., 24., 19., 31., 21.])

In [15]:
data["Location Description"].unique()

array(['STREET', 'APARTMENT', 'SIDEWALK', 'RESIDENCE PORCH/HALLWAY',
       'PARKING LOT/GARAGE(NON.RESID.)', 'CTA GARAGE / OTHER PROPERTY',
       'DRIVEWAY - RESIDENTIAL', 'GAS STATION', 'OTHER',
       'GROCERY FOOD STORE', 'ALLEY', 'BAR OR TAVERN',
       'VEHICLE NON-COMMERCIAL', 'RESIDENCE',
       'FACTORY/MANUFACTURING BUILDING', 'CTA BUS STOP',
       'SCHOOL, PUBLIC, BUILDING', 'DRUG STORE',
       'COMMERCIAL / BUSINESS OFFICE', 'RESIDENCE-GARAGE',
       'SMALL RETAIL STORE', 'NURSING HOME/RETIREMENT HOME',
       'VACANT LOT/LAND', 'RESTAURANT', 'HOSPITAL BUILDING/GROUNDS',
       'DEPARTMENT STORE', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP',
       'SCHOOL, PUBLIC, GROUNDS', 'HOTEL/MOTEL', 'MEDICAL/DENTAL OFFICE',
       'SCHOOL, PRIVATE, BUILDING', 'CTA BUS',
       'RESIDENTIAL YARD (FRONT/BACK)', 'GOVERNMENT BUILDING/PROPERTY',
       'DAY CARE CENTER', 'AIRPORT/AIRCRAFT', 'CAR WASH', 'BANK',
       'TAVERN/LIQUOR STORE', 'COLLEGE/UNIVERSITY RESIDENCE HALL',
       'COLLEGE/

#### Actually we can use tf.feature_column.categorical_column_with_hash_bucket to do it automatically

```
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
for i in data["Location Description"].unique():
    if "AIR" in i:
        a.append(i)
    elif "SCHOOL" in i or "UNIVERSITY" in i:
        b.append(i)
    elif "BUS" in i or "AUTO" in i or"TAXI" in i or"TRAIN"in i or"UBER" in i or"CTA"in i or"TRANSPORTATION" in i or"VEHICLE" in i or "MOTEL"in i or"TRUCK"in i:
        c.append(i)
    elif "STREET" in i or "SIDEWALK" in i or "WAY" in i or "GROUNDS" in i or "ALLEY" in i or "PARKING" in i or "FOREST" in i or "LAKE" in i or "RIVER" in i or "YARD" in i or "BRIDGE" in i or "FARM" in i or "SEWER" in i or "LAND" in i or "GARAGE" in i or "LAGOON" in i or "CEMETARY" in i:
        d.append(i)
    elif i=="OTHER" or i=="DUMPSTER" or i=='HORSE STABLE':
        e.append(i)
    else:
        f.append(i)
    ```

        

```
data["Location Description"].replace(a,'AIRPORT', inplace= True)
data["Location Description"].replace(b,'SCHOOL', inplace= True)
data["Location Description"].replace(c,'TRANSPORTATION', inplace= True)
data["Location Description"].replace(d,'OUTDOOR', inplace= True)
data["Location Description"].replace(e,'OTHER', inplace= True)
data["Location Description"].replace(f,'INDOOR', inplace= True)
```

In [16]:
data["Location Description"].unique()

array(['STREET', 'APARTMENT', 'SIDEWALK', 'RESIDENCE PORCH/HALLWAY',
       'PARKING LOT/GARAGE(NON.RESID.)', 'CTA GARAGE / OTHER PROPERTY',
       'DRIVEWAY - RESIDENTIAL', 'GAS STATION', 'OTHER',
       'GROCERY FOOD STORE', 'ALLEY', 'BAR OR TAVERN',
       'VEHICLE NON-COMMERCIAL', 'RESIDENCE',
       'FACTORY/MANUFACTURING BUILDING', 'CTA BUS STOP',
       'SCHOOL, PUBLIC, BUILDING', 'DRUG STORE',
       'COMMERCIAL / BUSINESS OFFICE', 'RESIDENCE-GARAGE',
       'SMALL RETAIL STORE', 'NURSING HOME/RETIREMENT HOME',
       'VACANT LOT/LAND', 'RESTAURANT', 'HOSPITAL BUILDING/GROUNDS',
       'DEPARTMENT STORE', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP',
       'SCHOOL, PUBLIC, GROUNDS', 'HOTEL/MOTEL', 'MEDICAL/DENTAL OFFICE',
       'SCHOOL, PRIVATE, BUILDING', 'CTA BUS',
       'RESIDENTIAL YARD (FRONT/BACK)', 'GOVERNMENT BUILDING/PROPERTY',
       'DAY CARE CENTER', 'AIRPORT/AIRCRAFT', 'CAR WASH', 'BANK',
       'TAVERN/LIQUOR STORE', 'COLLEGE/UNIVERSITY RESIDENCE HALL',
       'COLLEGE/

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6051090 entries, 0 to 6720962
Data columns (total 25 columns):
ID                      int64
Case Number             object
Date                    object
Block                   object
IUCR                    object
Primary Type            object
Description             object
Location Description    object
Arrest                  bool
Domestic                bool
Beat                    int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                object
X Coordinate            float64
Y Coordinate            float64
Year                    int64
Updated On              object
Latitude                float64
Longitude               float64
Location                object
date_parsed             datetime64[ns]
dayofweek               int64
hour                    int64
dtypes: bool(2), datetime64[ns](1), float64(7), int64(5), object(10)
memory usage: 1.1+ GB


In [18]:
data=data.set_index("date_parsed")

In [19]:
features=["Location Description","Arrest","Domestic","District","dayofweek","hour"]


In [ ]:
data.to_csv("data_clean")

# Task 2 - Convert the features in a form that can be given as input to tensorflow library/functions

Here you have to search a little bit. The data that we are using has data other than numerical values. Tensors take two kinds of values, either numeric or categorical. So, you have to convert the given data into a logical form which can be given to tensors.

In [20]:
len(data["Primary Type"].unique())

32

In [21]:
label=pd.get_dummies(data["Primary Type"])

In [22]:
X=data[features]

In [23]:
X=pd.get_dummies(X,columns=features)

In [24]:
X

,Location Description_ABANDONED BUILDING,Location Description_AIRCRAFT,Location Description_AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA,Location Description_AIRPORT BUILDING NON-TERMINAL - SECURE AREA,Location Description_AIRPORT EXTERIOR - NON-SECURE AREA,Location Description_AIRPORT EXTERIOR - SECURE AREA,Location Description_AIRPORT PARKING LOT,Location Description_AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA,Location Description_AIRPORT TERMINAL LOWER LEVEL - SECURE AREA,Location Description_AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
date_parsed,,,,,,,,,,,,,,,,,,,,,
2015-03-18 19:44:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2015-03-18 23:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2015-03-18 22:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2015-03-18 22:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2015-03-18 21:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2015-03-18 22:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2015-03-18 23:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2015-03-18 21:35:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2015-03-18 22:09:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, label, test_size=0.2, random_state=0)

# Task 3 - Build the model

Once the features and tensors are made, you have to build a machine learning model which, given the features predicts the type of crime. 
Report the accuracy of the model.

In [26]:
y_train=tf.cast(y_train, tf.float32)

In [27]:
#Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 3000000
display_step = 1

n_samples = X_train.shape[0]
n_features = 238
n_class = 32
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.zeros([n_features, n_class]))
b = tf.Variable(tf.zeros([n_class]))

pred = tf.matmul(x, W) + b
#define loss
cost = tf.losses.mean_squared_error(pred,y)


In [28]:
#Biuld my model
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.initialize_all_variables()


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [29]:
import timeit

In [30]:
%%time
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(n_samples / batch_size)
        for i in range(total_batch):
            _, c = sess.run([optimizer, cost], 
                            feed_dict={x: X_train[i * batch_size : (i+1) * batch_size], 
                                      y: y_train[i * batch_size : (i+1) * batch_size, :].eval()})
            avg_cost = c / total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", "%04d" % (epoch+1), "cost=", avg_cost.mean())

    print("Optimization Finished!")

    print("Testing Accuracy:", accuracy.eval({x: X_train, y:y_train.eval()}))


Epoch: 0001 cost= 0.03125
Epoch: 0002 cost= 0.03124997578561306
Epoch: 0003 cost= 0.03124990127980709
Epoch: 0004 cost= 0.03124972991645336
Epoch: 0005 cost= 0.031249620020389557
Epoch: 0006 cost= 0.03124943934381008
Epoch: 0007 cost= 0.03124893270432949
Epoch: 0008 cost= 0.031248822808265686
Epoch: 0009 cost= 0.031248481944203377
Epoch: 0010 cost= 0.031248031184077263
Optimization Finished!
Testing Accuracy: 0.27254242
CPU times: user 2min 50s, sys: 1min 5s, total: 3min 55s
Wall time: 1min 42s


# Task 4 - Run the model on your laptop and AWS cloud and observe the time taken for reading the data, training and testing. 

Report your observations. Due to more RAM needed for the same, it might be the case that it doesn't run on your laptop, so in that case report your laptop configuration and state it didnt run.

CPU times: user 2min 50s, sys: 1min 5s, total: 3min 55s

Wall time: 1min 42s

It didn't run on local jupyter notebook.

Laptop configuration：
MacBook Air (13-inch, Mid 2012)
1.8 GHz Intel Core i5
4 GB 1600 MHz DDR3

# Bonus Tasks

# Option 1: Try to build a Neural Network model, train on the same features and report the accuracy.
Report your observations.

In [31]:
import keras

Using TensorFlow backend.


In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

In [33]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(238, input_dim=238, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [34]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)


In [35]:
kfold = KFold(n_splits=5, random_state=seed)
results = cross_val_score(estimator, X, label, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

KeyboardInterrupt: 

#### It keep running without output.

In [ ]:
# define a Deeper model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(238, input_dim=238, kernel_initializer='normal', activation='relu'))
    model.add(Dense(120, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
np.random.seed(seed)
estimators = []
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=10, batch_size=5, verbose=0)))
estimators.fit(X_train, y_train)
#kfold = KFold(n_splits=5, random_state=seed)
#results = cross_val_score(estimators, X_train, y_train, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

# Option 2: Try to predict other things like Arrest or Description of Crime. 

You can change the features in this task. Report your observations.

In [5]:
label=pd.get_dummies(data["Arrest"])

In [6]:
features=["Location Description","Primary Type","Domestic","District","dayofweek","hour"]

In [7]:
X=data[features]
X=pd.get_dummies(X,columns=features)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, label, test_size=0.2, random_state=0)

In [10]:
y_train=tf.cast(y_train, tf.float32)

In [12]:
learning_rate = 0.001
training_epochs = 30
batch_size = 3000000
display_step = 1

n_samp = X_train.shape[0]
n_features = 268
n_class = 2
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.zeros([n_features, n_class]))
b = tf.Variable(tf.zeros([n_class]))

pred = tf.matmul(x, W) + b
cost = tf.losses.mean_squared_error(pred,y)

In [13]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [15]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(n_samp / batch_size)
        for i in range(total_batch):
            _, c = sess.run([optimizer, cost], 
                            feed_dict={x: X_train[i * batch_size : (i+1) * batch_size], 
                                      y: y_train[i * batch_size : (i+1) * batch_size, :].eval()})
            avg_cost = c / total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", "%04d" % (epoch+1), "cost=", avg_cost.mean())

    print("Optimization Finished!")

    print("Testing Accuracy:", accuracy.eval({x: X_train, y:y_train.eval()}))

Epoch: 0001 cost= 0.5
Epoch: 0002 cost= 0.4985041618347168
Epoch: 0003 cost= 0.4971243441104889
Epoch: 0004 cost= 0.49579986929893494
Epoch: 0005 cost= 0.494495153427124
Epoch: 0006 cost= 0.4932529926300049
Epoch: 0007 cost= 0.4918023645877838
Epoch: 0008 cost= 0.49047666788101196
Epoch: 0009 cost= 0.489123672246933
Epoch: 0010 cost= 0.48774054646492004
Epoch: 0011 cost= 0.4864112138748169
Epoch: 0012 cost= 0.4851187467575073
Epoch: 0013 cost= 0.4838709533214569
Epoch: 0014 cost= 0.4825797975063324
Epoch: 0015 cost= 0.4812357425689697
Epoch: 0016 cost= 0.4799253046512604
Epoch: 0017 cost= 0.47863534092903137
Epoch: 0018 cost= 0.47739917039871216
Epoch: 0019 cost= 0.47612351179122925
Epoch: 0020 cost= 0.47487300634384155
Epoch: 0021 cost= 0.4736074209213257
Epoch: 0022 cost= 0.4723545014858246
Epoch: 0023 cost= 0.4711160957813263
Epoch: 0024 cost= 0.469880610704422
Epoch: 0025 cost= 0.46864941716194153
Epoch: 0026 cost= 0.46739599108695984
Epoch: 0027 cost= 0.46614569425582886
Epoch: 00

## Amazing!